In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

import os
import re
import json
import time
import datetime
from tqdm import tqdm

import threading

In [2]:
from crawler import APIClient

In [3]:
api = APIClient()
print(api)

Chrome Webscraper <selenium.webdriver.chrome.webdriver.WebDriver (session="77eeaae4c337f36523cd0c8ace628cd3")> (C:\Users\dmarq\.wdm\drivers\chromedriver\win32\113.0.5672.63\chromedriver.exe)


In [ ]:
tmpdata = api.get_stock_data('tsla','2023-03-14',25,progress=True)

In [4]:
class apiThreadSlave(threading.Thread):
   def __init__(self, thread_id, tag, dates, num_links):
      threading.Thread.__init__(self)
      self.thread_id = thread_id
      self.tag = tag
      self.dates = dates
      self.num_links = num_links
      self.api = APIClient()
   def run(self):
      for i, date in enumerate(self.dates):
         tmpdata = self.api.get_stock_data(self.tag, date,self.num_links)
         if tmpdata.shape[0] > 0:
            tmpdata.to_csv(r'data/{}/{}.csv'.format(self.tag,date),index=False)
            print('Thread ID:',self.thread_id,'Link #',i)

In [9]:
#@title Multithreading
MAX_THREADS = 12
TAG = 'tsla'

dt_rng = pd.date_range(start="2022-01-01", end="2022-12-31").date
if os.listdir(f'data/{TAG}/') != []: # dont re-scrape dates already scraped for the given TAG
    dt_rng = list(set(dt_rng) - set(pd.to_datetime([x[0:-4] for x in os.listdir(f'data/{TAG}/')]).date))
# doesnt repeat days already done
n = len(dt_rng) // MAX_THREADS # number of dates per thread
thread_dt_rng = [dt_rng[x * n:(x + 1) * n] for x in range((len(dt_rng) + n - 1) // n )]

threads = [] 

for i, dates in enumerate(thread_dt_rng):
    tmp_thread = apiThreadSlave(i, TAG, dates, 50)
    tmp_thread.start()
    threads.append(tmp_thread)
    time.sleep(15) # wait 15 seconds between spawning threads to prevent google search throttling

for t in threads:
    t.join()

Error: MaxRecursionError https://twitter.com/IASJ_org/status/1555289985506582528
Thread ID: 0 Link # 0
Error: MaxRecursionError https://www.futurecar.com/tag/Tesla
Thread ID: 2 Link # 0
Thread ID: 1 Link # 0
Error: MaxRecursionError https://twitter.com/Tesla/status/1602789357156536321
Error: MaxRecursionError https://www.futurecar.com/tag/Tesla
Error: MaxRecursionError https://twitter.com/ClaraJeffery/status/1604889364978225152
Error: MaxRecursionError https://www.teslaworkerrights.org/press.html
Thread ID: 3 Link # 0
Thread ID: 0 Link # 1
Thread ID: 4 Link # 0


c:\Users\dmarq\miniconda3\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


Thread ID: 2 Link # 1
Error: MaxRecursionError https://investorplace.com/2022/02/google


c:\Users\dmarq\miniconda3\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


Error: MaxRecursionError https://www.travis.af.mil/SiteMap.aspx
Error: MaxRecursionError https://beta.unpri.org/system/files/supporting-materials/2022-07/22-07-12%20Proxy%20Alert%20-%20Tesla.docx
Thread ID: 5 Link # 0
Thread ID: 4 Link # 1
Thread ID: 3 Link # 1
Thread ID: 7 Link # 0
Thread ID: 2 Link # 2
Thread ID: 8 Link # 0
Error: MaxRecursionError https://twitter.com/tsrandall/status/1592191807592402944?lang=en
Error: MaxRecursionError https://eue.walz-offenburg.de/indiana-busted-newspaper.html
Thread ID: 1 Link # 1
Thread ID: 6 Link # 0
Error: MaxRecursionError https://www.teslaworkerrights.org/press.html
Thread ID: 4 Link # 2
Thread ID: 3 Link # 2
Error: MaxRecursionError https://www.nanovest.io/en/us-stocks/tsla/
Thread ID: 9 Link # 0
Error: MaxRecursionError https://www.futurecar.com/tag/%20China
Error: MaxRecursionError https://mobile.twitter.com/ashleygjovik/status/1511598701210337280
Thread ID: 0 Link # 2
Error: MaxRecursionError https://www.teslaworkerrights.org/press.html
T

c:\Users\dmarq\miniconda3\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


Error: MaxRecursionError https://www.teslaworkerrights.org/press.html
Thread ID: 0 Link # 3
Error: MaxRecursionError https://www.teslaworkerrights.org/press.html
Thread ID: 7 Link # 2
Error: MaxRecursionError https://cnevpost.com/2022/04/16/teslas-shanghai-plant-will-enter-closed-loop-production-on-april-17-report-says/
Thread ID: 10 Link # 1
Thread ID: 8 Link # 1
Error: MaxRecursionError https://www.travis.af.mil/SiteMap.aspx
Thread ID: 5 Link # 2
Thread ID: 3 Link # 3
Error: MaxRecursionError https://www.wholesaledoorparts.com/2018-tesla-model-x-100d-specifications-ff-47908311
Thread ID: 2 Link # 4
Thread ID: 12 Link # 1
Error: MaxRecursionError https://fgijmyhxl.unifax.pl/
Error: MaxRecursionError https://blogs.gartner.com/avivah-litan/2022/04/13/tesla-gets-7-8m-in-real-estate-financing-using-makerdao-real-world-assets-meet-defi/
Thread ID: 6 Link # 2
Error: MaxRecursionError https://twitter.com/SawyerMerritt/status/1487244771206832130?lang=en
Thread ID: 0 Link # 4
Error: MaxRecursi

In [ ]:
# for c in os.listdir('data/tsla/'):
#     f = pd.read_csv(f'data/tsla/{c}')
#     f.drop(columns=['Unnamed: 0'],inplace=True)
#     f.to_csv(f'data/tsla/{c}',index=False)
# fixed the save index error from legacy run to normalize data format

# TESTING

In [ ]:
#url = "https://www.bloomberg.com/news/articles/2023-02-17/summers-says-too-soon-to-call-for-march-50-basis-point-fed-hike?srnd=premium"
#url = "https://www.bloomberg.com/news/articles/2023-02-18/cars-tires-textile-factories-have-shut-in-crisis-hit-pakistan?srnd=industries-v2"

testlinks = api.get_google_links('tsla','2022-02-01',25)
testlinks

In [ ]:
res = api.get_article_data(testlinks[4])
res

In [ ]:
html = api.get_html_from_url(testlinks[0])

In [ ]:
soup = BeautifulSoup(html, 'html.parser')
text = ''
for tmp in [tmp.text for tmp in soup.find_all(['a','p','h','h1','h2'])]:
    if len(tmp) > 25 and re.findall('Click here',tmp) == [] and re.findall('This Simple Trick',tmp) == []:
        text += tmp + ' '
text = re.sub('\s+',' ',text.replace('\n',' ').replace('\xa0',' ').replace('\'','’').replace('   ',' ').strip())
text